In [7]:
import os

while os.getcwd().split("/")[-1] != "alfa-hack-rag":
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [8]:
import pandas as pd

In [ ]:
reference_df = pd.read_csv("ranking_results/submit_28,2.csv")
test_df = pd.read_csv("ranking_results/submit_user2_small_25.csv")

In [10]:
def parse_web_list(web_list_series):
    return web_list_series.str[1:-1].str.split(", ").apply(lambda x: [int(num) for num in x])

In [11]:
reference_df["web_list"] = parse_web_list(reference_df["web_list"])
test_df["web_list"] = parse_web_list(test_df["web_list"])

In [12]:
def calculate_hit_at_k(reference_df, test_df, k=5):
    """
    Рассчитывает метрику Hit@k и возвращает DataFrame с детальными результатами

    Parameters:
    reference_df: DataFrame с эталонными данными
    test_df: DataFrame с предсказанными данными
    k: количество top результатов для проверки

    Returns:
    tuple: (hit_at_k, results_df) где results_df содержит детальную информацию по каждому вопросу
    """
    
    results = []
    valid_questions = 0
    total_hits = 0

    # Проходим по всем вопросам в тестовом DataFrame
    for _, test_row in test_df.iterrows():
        q_id = test_row["q_id"]
        test_webs = test_row["web_list"][:k]  # Берем первые k элементов

        # Находим соответствующий вопрос в эталонном DataFrame
        reference_row = reference_df[reference_df["q_id"] == q_id]

        # Если вопрос есть в эталонных данных
        if not reference_row.empty:
            reference_webs = reference_row.iloc[0]["web_list"]

            # Проверяем, есть ли хотя бы одно пересечение
            intersection = set(test_webs) & set(reference_webs)
            hit = 1 if intersection else 0
            
            total_hits += hit
            valid_questions += 1

            # Сохраняем детальную информацию
            results.append({
                'q_id': q_id,
                'reference_webs': reference_webs,
                'test_webs': test_webs,
                'intersection': list(intersection),
                'intersection_count': len(intersection),
                'hit@k': hit
            })

    # Создаем DataFrame с результатами
    results_df = pd.DataFrame(results)
    
    # Рассчитываем итоговую метрику
    if valid_questions > 0:
        hit_at_k = total_hits / valid_questions
    else:
        hit_at_k = 0

    return hit_at_k, results_df

# Пример использования:
hit_at_5, results_df = calculate_hit_at_k(reference_df, test_df, k=5)
print(f"Hit@5 = {hit_at_5:.4f}")
results_df

Hit@5 = 0.5244


,q_id,reference_webs,test_webs,intersection,intersection_count,hit@k
0,1,"[372, 108, 1157, 789, 1098]","[1157, 372, 1896, 593, 1098]","[1098, 372, 1157]",3,1
1,2,"[372, 368, 1157, 1098, 1609]","[1157, 1098, 1080, 135, 368]","[368, 1098, 1157]",3,1
2,3,"[116, 1915, 1147, 1067, 857]","[1900, 116, 114, 1902, 110]",[116],1,1
3,4,"[1043, 478, 1038, 1924, 1552]","[796, 1043, 1006, 1900, 1902]",[1043],1,1
4,5,"[1029, 165, 1040, 1586, 1025]","[1193, 1029, 1034, 1025, 156]","[1025, 1029]",2,1
...,...,...,...,...,...,...
6972,6973,"[1902, 1552, 716, 1915, 420]","[1902, 420, 716, 1197, 1516]","[716, 420, 1902]",3,1
6973,6974,"[372, 1627, 1905, 1527, 1554]","[1905, 1627, 66, 1527, 86]","[1905, 1627, 1527]",3,1
6974,6975,"[1915, 1014, 1202, 410, 692]","[886, 1042, 1193, 796, 1607]",[],0,0
6975,6976,"[819, 108, 688, 1072, 1157]","[1567, 16, 1072, 1896, 1125]",[1072],1,1
